In [ ]:
import numpy as np
import random

def softmax(x):
    e_x = np.exp(x)
    return e_x / e_x.sum(axis=0)

def smooth(loss, cur_loss):
    return loss*0.999 + cur_loss*0.001

def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]
    print('%s' % (txt,), end='')
    
def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length

def initialize_params(n_a, n_x, n_y):
    np.random.seed(1)
    Wax = np.random.randn(n_a,n_x)*0.01
    Waa = np.random.randn(n_a,n_a)*0.01
    Wya = np.random.randn(n_y,n_a)*0.01
    b = np.zeros((n_a,1))
    by = np.zeros((n_y,1))
    
    params = {"Wax":Wax, "Waa":Waa, "Wya":Wya, "b":b, "by":by}
    return params

def rnn_step_backward(dy, gradients,params, x,a,a_prev):
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(params['Wya'].T, dy) + gradients['da_next']
    daraw = (1-a*a)*da
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(params['Waa'].T, daraw)
    return gradients

def update_params(params, gradients, lr):
    params['Wax'] += -lr * gradients['dWax']
    params['Waa'] += -lr * gradients['dWaa']
    params['Wya'] += -lr * gradients['dWya']
    params['b'] += -lr * gradients['db']
    params['by'] += -lr * gradients['dby']
    return params

def rnn_forward(X,Y,a0,params,vocab_size=27):
    x, a, y_hat = {},{},{}
    